In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from sklearn.decomposition import PCA
from sklearn.datasets import fetch_olivetti_faces
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder

APARTADO A

In [3]:
X, y = fetch_olivetti_faces(return_X_y=True);

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, random_state=1)

In [5]:
X_train = X_train / np.max(X)
X_test = X_test / np.max(X)

In [6]:
encoder = OneHotEncoder(sparse_output=False)
y_train = encoder.fit_transform(y_train.reshape(-1, 1))
y_test = encoder.fit_transform(y_test.reshape(-1, 1))

APARTADO B

In [7]:
def train_models(train, test, size, neuronas, b):

    pred_res = []

    for i in neuronas:
        # Crear modelo, si b es true hacemos el apartado b sino hacemos el apartado c
        if b:
            model = keras.Sequential(name=  str(size) +'_componentes_'+ str(i)+'_neuronas')
            model.add(keras.Input(shape=size))
            model.add(keras.layers.Dense(i, activation="relu", name='Dense_1'))
            model.add(keras.layers.Dense(40, activation="softmax", name='Dense_2'))
        else:
            model = keras.Sequential(name='Capas_convolucionales')
            model.add(keras.Input(shape=(64,64,1,)))
            model.add(keras.layers.Conv2D(filters=i, kernel_size=3, strides=1, activation="relu", name='Conv2D'))
            model.add(keras.layers.Flatten(name='Flatten'))
            model.add(keras.layers.Dense(40, activation="softmax", name='dense_1'))

        # Compilamos
        model.compile(optimizer=keras.optimizers.SGD(), loss=tf.keras.losses.CategoricalCrossentropy())
        model.fit(train,y_train,batch_size=16, epochs=200,verbose=False)

        # Guardamos la predicción
        pred = []
        pred = model.predict(test)

        pred_res.append(pred.argmax(axis=1))

    return pred_res, model

In [8]:
def compute_accuracies(predictions, true_labels):
    accuracies = []
    for i in range(len(predictions)):
        correct_elem = np.sum(true_labels == predictions[i])
        accuracies.append((i, correct_elem / len(predictions[i])))

    return accuracies

He creado estas 2 funciones que nos ayudaran a lo largo de los apartados, la primera es una funcion que entrena modelos y devuelve su capacidad de predecir y el modelo final, la segunda coge la capacidad de predecir y la etiqueta original y comprueba cual es la tasa de acierto del modelo.

In [9]:
neuronas = [25, 50, 75, 100]

pca_10 = PCA(n_components=10)
pca_10.fit(X_train);

X_train_10 = pca_10.transform(X_train)
X_test_10 = pca_10.transform(X_test)

pred_res_10, model_10_100 = train_models(X_train_10, X_test_10, 10, neuronas, 1)

PCA(n_components=10)



4/4 [==============================] - 0s 2ms/step


In [10]:
pca_20 = PCA(n_components=20)
pca_20.fit(X_train);

X_train_20 = pca_20.transform(X_train)
X_test_20 = pca_20.transform(X_test)

pred_res_20, model_20_100 = train_models(X_train_20, X_test_20, 20, neuronas, 1)

PCA(n_components=20)

4/4 [==============================] - 0s 2ms/step


In [11]:
pred_res = pred_res_10 + pred_res_20
accuracy = compute_accuracies(pred_res, y_test.argmax(axis=1) + 1)

print('Neuronas 25 50 75 100 con PCA de 10 componentes:')
for x in accuracy:
    if x[0] == 4:
        print('\nNeuronas 25 50 75 100 con PCA de 20 componentes:')
    print(x[1])

Neuronas 25 50 75 100 con PCA de 10 componentes:
0.65
0.675
0.6916666666666667
0.725

Neuronas 25 50 75 100 con PCA de 20 componentes:
0.725
0.775
0.7916666666666666
0.8


El uso de un modelo PCA de 20 componentes es mejor que el modelo con 10 componentes, ahora bien, tras realizar pruebas varias veces el resultado de usar entre 75 a 100 neuronas va variando probablemente en función a momento de uso de la GPU, pero el resultado va variando alrededor del 0.8 por lo tanto sera el que utilizemos para guiarnos en la practica.

Como he comentado antes, el maximo potencial se adquiere entre las 75 a 100 neuronas, por lo tanto no se puede llegar a la conclusión que mas neuronas influya a una mejor tasa de acierto. Cosas como la complejidad del problema, tamaño del conjunto de datos y otros factores pueden ser determinantes a la hora de elegir el numero de neuronas.

APARTADO C

In [12]:
X_train = X_train.reshape(-1,64,64,1);
X_test = X_test.reshape(-1,64,64,1);

In [13]:
neuronas = np.arange(1,11)
pred_res_cnn, model_cnn = train_models(X_train, X_test, 0, neuronas, 0)

4/4 [==============================] - 0s 4ms/step


In [14]:
accuracies_cnn = compute_accuracies(pred_res_cnn, y_test.argmax(axis=1) + 1)

for i in range(0,10):
  print('Neuronas', i+1, ':', accuracies_cnn[i][1])

Neuronas 1 : 0.875
Neuronas 2 : 0.8916666666666667
Neuronas 3 : 0.8833333333333333
Neuronas 4 : 0.8833333333333333
Neuronas 5 : 0.8916666666666667
Neuronas 6 : 0.8916666666666667
Neuronas 7 : 0.8916666666666667
Neuronas 8 : 0.8916666666666667
Neuronas 9 : 0.8916666666666667
Neuronas 10 : 0.8916666666666667


Desde el principio la tasa de acierto es superior al anterior usando PCA, sacando casi 0.1 de acierto respecto al mejor modelo anterior.

Para el ultimo apartado utilizaremos el modelo de 10 neuronas con un acierto de 0.89 ya que a simple vista parece que rinde un poco peor pero como he comentado antes esto puede ser debido al rendimiento de la CPU, y al haber ejecutado varias veces se puede observar que llega a dicha tasa regularmente.

APARTADO D

In [15]:
model_10_100.summary()

Model: "10_componentes_100_neuronas"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 100)               1100      
                                                                 
 Dense_2 (Dense)             (None, 40)                4040      
                                                                 
Total params: 5140 (20.08 KB)
Trainable params: 5140 (20.08 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [16]:
model_20_100.summary()

Model: "20_componentes_100_neuronas"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Dense_1 (Dense)             (None, 100)               2100      
                                                                 
 Dense_2 (Dense)             (None, 40)                4040      
                                                                 
Total params: 6140 (23.98 KB)
Trainable params: 6140 (23.98 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [17]:
model_cnn.summary()

Model: "Capas_convolucionales"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Conv2D (Conv2D)             (None, 62, 62, 10)        100       
                                                                 
 Flatten (Flatten)           (None, 38440)             0         
                                                                 
 dense_1 (Dense)             (None, 40)                1537640   
                                                                 
Total params: 1537740 (5.87 MB)
Trainable params: 1537740 (5.87 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [18]:
model = keras.Sequential(name='red MLP')
model.add(keras.Input(shape=(64,64)))
model.add(keras.layers.Dense(units=100, activation="relu", name='dense_1'))
model.add(keras.layers.Dense(40, activation="softmax", name='dense_2'))

model.summary()

Model: "red MLP"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_1 (Dense)             (None, 64, 100)           6500      
                                                                 
 dense_2 (Dense)             (None, 64, 40)            4040      
                                                                 
Total params: 10540 (41.17 KB)
Trainable params: 10540 (41.17 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Como podemons observar disponemos de 4 modelos, el primero tenia un rendimiento maximo de 0.73 mientras el segundo siendo un PCA de 20 componentes su maximo rendimiento era de 0.8, en cambio el modelo utilizado en el apartado C llegaba casi a los 0.9.

En cambio podemos observar un claro aumento de la cantidad de parametro a entrenar, siendo 6140 parametros en el mejor modelo del apartado B debido a la reducción de tamaño del PCA y 1537740 parametros en el apartado C probablemante debido al añadir una nueva dimensión y hacerla una matriz cuadrada.

Por lo tanto a la hora de elegir que modelo es mejor deberemos pensar entre coste computacional y rendimiento (apartado B) frente a capacidad de acierto (apartado C).

Respecto a la red MLP este utilizaria 10540 parametros respecto a los 6140 del segundo modelo debido a que no ha habido reducción habiendo 6500 parametros en la primera capa en comparación a 2100 parametros, siendo importante el uso de componentes PCA para computar el modelo. 